In [1]:
import os
import pandas as pd

In [8]:
pwd = os.getcwd()

filepath = pwd + '/Uncleaned_DS_jobs.csv'

raw_data = pd.read_csv(filepath) 

wip = raw_data.copy()

In [22]:
# replace \n with delimiter
wip['Company Name'] = wip['Company Name'].str.replace("\n", ",", regex=False)

# remove everything after delimiter
result = []
for item in wip['Company Name']:
    parts = item.split(',')
    result.append(','.join(parts[:1]))

wip['Company Name'] = result

In [50]:
# remove $
wip['SalaryEstUSD'] = wip['Salary Estimate'].str.replace('$', '', regex=False)

# add delimiter
# two formats present '''$145K-$225K (Glassdoor est.)''' and '''$145K-$225K(Employer est.)'''
wip['SalaryEstUSD'] = wip['SalaryEstUSD'].str.replace('(', ':', regex=False)

# remove everything after delimiter
result = []
for item in wip['SalaryEstUSD']:
    parts = item.split(':')
    result.append(':'.join(parts[:1]))

wip['SalaryEstUSD'] = result

In [52]:
# split into new columns
wip[['MinSalaryEstUSD', 'MaxSalaryEstUSD']] = wip['SalaryEstUSD'].str.split("-", expand=True)

# convert values
def convert_values(val):
    if 'K' in val:
        return float(val.replace('K', '')) * 1000
    else:
        return float(val)

wip['MinSalaryEstUSD'] = wip['MinSalaryEstUSD'].apply(convert_values)
wip['MaxSalaryEstUSD'] = wip['MaxSalaryEstUSD'].apply(convert_values)

# convert from float to int
wip['MinSalaryEstUSD'] = wip['MinSalaryEstUSD'].astype(int)
wip['MaxSalaryEstUSD'] = wip['MaxSalaryEstUSD'].astype(int)

In [ ]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

In [80]:
# Function to tokenize and count words
def count_words(text):
    # Tokenize words
    tokens = nltk.word_tokenize(text)
    
    # Filter out stopwords and non-alphabetic tokens
    words = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
    
    # Count word frequencies
    word_counts = Counter(words)
    
    return word_counts

# Apply function to 'job_description' column
wip['Word Count'] = wip['Job Description'].apply(count_words)

# Aggregate word counts across all job descriptions
total_word_counts = Counter()
for counts in wip['Word Count']:
    total_word_counts += counts

print(word_counts.most_common(100))
for word, frequency in most_common_words:
    print(f"{word}: {frequency}")

[('and', 19116), ('to', 10405), ('the', 8040), ('data', 7370), ('of', 7295), ('in', 5760), ('a', 5632), ('with', 5052), ('for', 3612), ('experience', 3395), ('or', 3231), ('our', 2510), ('is', 2334), ('we', 2319), ('you', 2268), ('as', 2007), ('on', 1766), ('will', 1650), ('work', 1564), ('science', 1519), ('are', 1518), ('that', 1490), ('learning', 1480), ('business', 1424), ('an', 1370), ('team', 1369), ('be', 1364), ('s', 1289), ('machine', 1231), ('skills', 1209), ('at', 1024), ('have', 1009), ('analysis', 992), ('this', 986), ('from', 937), ('analytics', 925), ('development', 917), ('models', 888), ('ability', 887), ('by', 841), ('other', 840), ('years', 815), ('solutions', 812), ('engineering', 779), ('tools', 760), ('knowledge', 756), ('technical', 745), ('scientist', 739), ('company', 735), ('your', 717), ('working', 716), ('using', 699), ('systems', 694), ('information', 673), ('support', 667), ('new', 662), ('job', 653), ('python', 646), ('strong', 642), ('software', 639), ('

In [78]:
wip['Word Count']

0      {'description': 3, 'senior': 1, 'data': 10, 's...
1      {'secure': 1, 'nation': 1, 'ignite': 1, 'futur...
2      {'overview': 1, 'analysis': 3, 'group': 3, 'on...
3      {'job': 3, 'description': 1, 'passion': 1, 'da...
4      {'data': 8, 'scientist': 1, 'affinity': 1, 'so...
                             ...                        
667    {'summary': 1, 'looking': 3, 'data': 20, 'scie...
668    {'job': 1, 'description': 1, 'become': 1, 'tho...
669    {'join': 2, 'thriving': 1, 'company': 1, 'chan...
670    {'remote': 1, 'opportunity': 1, 'ainlp': 1, 'd...
671    {'description': 1, 'data': 7, 'scientist': 1, ...
Name: Word Count, Length: 672, dtype: object

In [81]:
wip

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Salary Est,SalaryEstUSD,MinSalaryEstUSD,MaxSalaryEstUSD,Word Count
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137K-171K (Glassdoor est.),137K-171K,137000,171000,"{'description': 3, 'senior': 1, 'data': 10, 's..."
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,137K-171K (Glassdoor est.),137K-171K,137000,171000,"{'secure': 1, 'nation': 1, 'ignite': 1, 'futur..."
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1,137K-171K (Glassdoor est.),137K-171K,137000,171000,"{'overview': 1, 'analysis': 3, 'group': 3, 'on..."
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",137K-171K (Glassdoor est.),137K-171K,137000,171000,"{'job': 3, 'description': 1, 'passion': 1, 'da..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",137K-171K (Glassdoor est.),137K-171K,137000,171000,"{'data': 8, 'scientist': 1, 'affinity': 1, 'so..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Data Scientist,$105K-$167K (Glassdoor est.),Summary\n\nWe’re looking for a data scientist ...,3.6,TRANZACT,"Fort Lee, NJ","Fort Lee, NJ",1001 to 5000 employees,1989,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1,105K-167K (Glassdoor est.),105K-167K,105000,167000,"{'summary': 1, 'looking': 3, 'data': 20, 'scie..."
668,668,Data Scientist,$105K-$167K (Glassdoor est.),Job Description\nBecome a thought leader withi...,-1.0,JKGT,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1,105K-167K (Glassdoor est.),105K-167K,105000,167000,"{'job': 1, 'description': 1, 'become': 1, 'tho..."
669,669,Data Scientist,$105K-$167K (Glassdoor est.),Join a thriving company that is changing the w...,-1.0,AccessHope,"Irwindale, CA",-1,-1,-1,-1,-1,-1,-1,-1,105K-167K (Glassdoor est.),105K-167K,105000,167000,"{'join': 2, 'thriving': 1, 'company': 1, 'chan..."
670,670,Data Scientist,$105K-$167K (Glassdoor est.),100 Remote Opportunity As an AINLP Data Scient...,5.0,ChaTeck Incorporated,"San Francisco, CA","Santa Clara, CA",1 to 50 employees,-1,Company - Private,Advertising & Marketing,Business Services,$1 to $5 million (USD),-1,105K-167K (Glassdoor est.),105K-167K,105000,167000,"{'remote': 1, 'opportunity': 1, 'ainlp': 1, 'd..."
